# AVITO PROJECT

## Team Member

* Sandikha Rahardi - РИМ 130908

## Load Data

### Import Package

In [372]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split

### Set Dataframe

In [373]:
dataset_filename = "assets/avito_dataset.xlsx"
main_df = pd.read_excel(dataset_filename, sheet_name="Sheet1")

## Data Cleaning

### Important Column

In [374]:
# Define important column to predict the price
important_columns = [
    'Цена', 'Метро', 'Метро2', 'Метро. Время', 
    'Район', 'Продавец. Компания', 'Статус', 'Количество комнат',
    'Общая площадь, число', 'Этаж', 'Санузел', 
    'Балкон или лоджия', 'Окна', 'Ремонт', 
    'Тип дома', 'Этажей в доме', 'Парковка', 
    'Двор', 'Грузовой лифт', 'В доме',
    'Тёплый пол', 'Дополнительно', 'Отделка',
    'Площадь кухни, число', 'Тип комнат'
]
print("Total original column : " , main_df.columns.__len__())
print("Total important column : " , important_columns.__len__())
main_df = main_df[important_columns]

Total original column :  71
Total important column :  25


In [375]:
print(main_df.shape)
main_df.isnull().sum() / main_df.shape[0] * 100

(19803, 25)


Цена                     0.000000
Метро                    0.489825
Метро2                  47.856385
Метро. Время            47.856385
Район                   52.310256
Продавец. Компания       0.000000
Статус                   0.000000
Количество комнат        0.000000
Общая площадь, число     0.000000
Этаж                     0.000000
Санузел                 40.301974
Балкон или лоджия       32.692016
Окна                    56.824724
Ремонт                  60.793819
Тип дома                13.750442
Этажей в доме            0.000000
Парковка                55.769328
Двор                    59.041559
Грузовой лифт           66.696965
В доме                  89.577337
Тёплый пол              97.136797
Дополнительно           92.637479
Отделка                 43.877190
Площадь кухни, число    19.491996
Тип комнат              54.128162
dtype: float64

### Clean the dataset

#### Remove duplicated dataset

In [376]:
print("Duplicated data : ", main_df.duplicated().sum())
main_df = main_df.drop_duplicates().reset_index(drop=True)
print("Duplicated data (now) : ", main_df.duplicated().sum())

Duplicated data :  257
Duplicated data (now) :  0


#### Clean about 'Метро'

In [377]:
main_df.dropna(subset=['Метро'], inplace=True)

def preprocess_metro_time(row):
    if(type(row) == str):
        row = row.replace("мин.","").strip()
        if(row.__contains__("от")):
            row = float(row.replace("от","").strip())
        elif(row.__contains__("до")):
            row = float(row.replace("до","").strip())
        elif(row.__contains__("–")):
            row = (int(row.split("–")[0]) + int(row.split("–")[1])) / 2
    return row

main_df['Метро. Время'] = main_df['Метро. Время'].apply(preprocess_metro_time)
mean_metro_time = round(pd.to_numeric(main_df['Метро. Время']).mean(), 1)
main_df['Метро. Время'].fillna(mean_metro_time, inplace=True)
main_df['Метро2'] = main_df['Метро2'].apply(lambda x: "Нет" if pd.isna(x) else "Есть")

#### Clean about 'Район'

In [378]:
main_df['Район'].fillna("Другой", inplace=True)

#### Clean about 'Количество комнат'

In [379]:
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: 0 if x == 'Студия' else x)

max_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').max()
mean_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').mean()

main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: max_room if x == 'многокомнатные' else x)
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: round(mean_room) if x == 'Своб. планировка' else x)

main_df['Количество комнат'].value_counts()

Количество комнат
2.0    6522
1.0    6279
3.0    3596
0.0    2475
4.0     493
5.0      67
6.0       9
7.0       8
Name: count, dtype: int64

> Note: Студия does not have room ( 0 room )

#### Clean about 'Общая площадь, число'

In [380]:
main_df['Общая площадь, число'] = main_df['Общая площадь, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else int(x))

#### Clean about 'Этаж'

In [381]:
# Clean dataset 'Этаж' of 'Этажей в доме'
main_df['Этаж'] = main_df['Этаж'].apply(lambda x: int(x.split("из")[0].strip()))

#### Clean about 'Санузел'

In [382]:
main_df['Санузел'].fillna('Другой', inplace=True)

#### Clean about 'Балкон или лоджия'

In [383]:
main_df['Балкон или лоджия'].fillna('Другой', inplace=True)

#### Clean about 'Окна'

In [384]:
main_df['Окна'].fillna('Другой', inplace=True)

#### Clean about 'Ремонт'

In [385]:
main_df['Ремонт'].fillna('Другой', inplace=True)

#### Clean about 'Тип дома'

In [386]:
main_df['Тип дома'].fillna('Другой', inplace=True)

#### Clean about 'Парковка'

In [387]:
main_df['Парковка'].fillna(0, inplace=True)

def preprocess_parking(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Парковка'] = main_df['Парковка'].apply(preprocess_parking)

#### Clean about 'Двор'

In [388]:
main_df['Двор'].fillna(0, inplace=True)

def preprocess_yard(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Двор'] = main_df['Двор'].apply(preprocess_yard)

#### Clean about 'Грузовой лифт'

In [389]:
# Change null value with mode
main_df['Грузовой лифт'] = main_df['Грузовой лифт'].apply(lambda x: 0 if x == 'Нет' else x)
print("Data Null :" , main_df['Грузовой лифт'].isna().sum())
print("Data Mode :" , main_df['Грузовой лифт'].mode()[0])
main_df['Грузовой лифт'].fillna(main_df['Грузовой лифт'].mode()[0], inplace=True)

Data Null : 12924
Data Mode : 1.0


#### Clean about 'В доме'

In [390]:
main_df['В доме'].fillna('Другой', inplace=True)

#### Clean about 'Тёплый пол'

In [391]:
main_df['Тёплый пол'].fillna('Нет', inplace=True)

#### Clean about 'Дополнительно'

In [392]:
main_df['Дополнительно'].fillna('Другой', inplace=True)

#### Clean about 'Отделка'

In [393]:
main_df['Отделка'].fillna('Другой', inplace=True)

#### Clean about 'Площадь кухни, число'

In [394]:
main_df['Площадь кухни, число'] = main_df['Площадь кухни, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else x)
print("Data Null :" , main_df['Площадь кухни, число'].isna().sum())
print("Data Mode :" , main_df['Площадь кухни, число'].mode()[0])
main_df['Площадь кухни, число'].describe()

Data Null : 3648
Data Mode : 6.0


count    15801.000000
mean        14.012847
std          7.045744
min          2.000000
25%          8.300000
50%         13.900000
75%         18.000000
max        100.000000
Name: Площадь кухни, число, dtype: float64

In [395]:
rounded_number = round(main_df['Площадь кухни, число'].mode()[0], 1)
main_df['Площадь кухни, число'].fillna(rounded_number, inplace=True)

#### Clean about 'Тип комнат'

In [396]:
main_df['Тип комнат'].fillna('Другой', inplace=True)

### Split Dataset

In [397]:
# Split the data into training (80%), testing (15%), and validation (5%)
train_df, temp_df = train_test_split(main_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.25, random_state=42)
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(15559, 25)
(2917, 25)
(973, 25)


### Divided into Categorical and Numerical dataset

In [398]:
numerical_columns = ['Цена', 'Метро. Время', 'Количество комнат', 'Общая площадь, число', 'Этаж', 'Этажей в доме', 'Парковка', 'Двор', 'Грузовой лифт', 'Площадь кухни, число']
categorical_columns = ['Метро', 'Метро2', 'Район', 'Продавец. Компания', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Тип дома', 'В доме', 'Тёплый пол', 'Дополнительно', 'Отделка', 'Тип комнат']

train_num_df = train_df[numerical_columns]
train_cat_df = train_df[categorical_columns]
test_num_df = test_df[numerical_columns]
test_cat_df = test_df[categorical_columns]
val_num_df = val_df[numerical_columns]
val_cat_df = val_df[categorical_columns]